# The purpose of this notebook is to analyze the oscilloscope data and spectrometer data to find the Finesse and Cavity Length 

In [ ]:
%matplotlib inline
import os
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt
c=3.e8 #speed of light
import time
filepath='D:/measuring/'
import scipy.signal
os.chdir(filepath)

sys.path.append('H:/My Documents/measuring/')

#Directory for the Spectrometer Data
data_dir='D:\measuring\data/20160824'#'D:\measuring\data/20160503\ON diamond\pos 7/raw data spectrometer\L9'#'D:\measuring\data/20160707\CL2_newmirror_processed'#"/Users/suzannevandam/Documents/PhD/localdata/20160430/ON_diamond" 
print data_dir

#Saving Directory
#save_dir=data_dir#'D:\measuring\data/20160805\Data_Summary'

%matplotlib inline

In [ ]:
import analysis.scripts.cavity.spectrometer_analysis as sa
import analysis.scripts.cavity.spectrometer_analysis_2Dsweep as sa_2D
import analysis.scripts.cavity.oscilloscope_analysis_linewidth as oscillo_ana

# We first analyze the linewidth of the cavity using the oscilloscope data

In [ ]:
#This fits the oscilloscope data to three linewidths 
#note that if you change the spacing between the side peaks (in GHz), 
#you must go to the oscilloscope_analysis_linewidth script in sublime and change it there 
reload(oscillo_ana)
#Directory for the Oscilloscope Data 
indir='D:\measuring\data/20160824/Scope_RF_ON_camera_up'#'D:\measuring\data/20160430\OFF_diamond'
filename='NICE_LWS00106'
EOM_freq=6.9 #GHz

#Selecting the correct peak
x_min = 16750 #Left most point of the x (in ms)
x_max = 20500 #Right most point in x (in ms)

oa = oscillo_ana.oscilloscope_analysis(indir=indir,filename=filename)
oa.get_data(use_timetrace=False,x_min=x_min,x_max=x_max)#1400)

#Fit parameters 
g_a1 = 0 #Offset in the y-axis (usually remains at 0)
g_A1 = 20 # Area of the large peaks ((High time-Low time)*Intensity)
g_gamma1 = 150 #Estimate of linewidth of the center peak (Width at half max (in ms))
#g_gamma2 = 200
g_dx = 520 #Difference in ms between the center peak and the side peaks
g_dx2 = 500
g_A2 = 4 #Area of the side peak on the left
#g_A3 = 2 #Area of the side peak on the right (only necessary if you are using the asymmetric peak fit)

g_x01 = x_min+(x_max-x_min)/2. #Automatically finds the maximum of the middle peak. You have to place the middle peak in the center of the image for this to get a good estimate. Alternatively you can just input g_x01 manually if you have multiple peaks. 
#g_x01=22250

fixed =[]

print g_x01

oa.plot_data()
#oa.plot_and_fit_with_EOM(EOM_freq=EOM_freq,show_fit=True,g_a1=g_a1, g_A1=g_A1, g_x01=g_x01, g_gamma1=g_gamma1, g_dx=g_dx, g_A2=g_A2, g_A3=g_A3, fixed=fixed)
oa.plot_and_fit_birefringence_with_EOM(EOM_freq=EOM_freq,show_fit=True,g_a1=g_a1, g_A1=g_A1, g_x01=g_x01, g_gamma1=g_gamma1, g_dx=g_dx,g_dx2=g_dx2 ,g_A2=g_A2, fixed=fixed)

# This code is for average multiple pieces of data and then finding the fit of the average 

# # lws_old=lws[:-1]
# lws = np.array([])#lws_old## 
# nr_lws=120

# x0 =860
# windowsize=300# print len(x)
# # xi = int(len(x)/nr_lws)
# # print xi 

# x,y = oa.get_data(use_timetrace=False)
# # lw,u_lw = oa.plot_and_fit_with_EOM(EOM_freq=EOM_freq)
# # print lw

# for i in np.arange(nr_lws):
#     xi = int(x0+(len(x)*i/nr_lws))
    
#     if i%2==1:
#         xi=xi-50   
#     if (xi-windowsize)>0:
#         x_min = xi-windowsize
#     else: 
#         x_min = 0
#     if (xi+windowsize)<len(x)-1:
#         x_max = xi+windowsize
#     else:
#         x_max =-1
#     oa.get_data(use_timetrace=False,x_min = x_min, x_max=x_max)
#     #oa.plot_data()
#     lw,u_lw = oa.plot_and_fit_with_EOM(EOM_freq=EOM_freq,g_gamma1=20, g_dx=60,g_A2=2,g_A3=2,show_fit=True)
#     lws = np.append(lws,lw)
# #     if i>1:
# #         break
        

# Now we analyze the spectrometer data to find the FSR/cavity length

In [ ]:
#note that what are called wavelengths, are actually the frequencies in THz! 
wavelengths,filenumbers,intensities = sa.load_data_from_folder(data_dir)
# wavelengths,filenumbers,intensities = sa.load_data(data_file)

print data_dir

In [ ]:
#Showing all of the peaks from the spectrometer data
print len(wavelengths)
plt.plot(wavelengths,intensities[:,0])
plt.ylabel('Intensity (a.u)')
plt.xlabel('Frequency (THz)')
plt.title('All Peaks - Spectrometer Data '+data_dir)
plt.savefig(os.path.join(data_dir, 'Full Spectrum - Spectrometer Data.jpg'))

In [ ]:
#Zooming in on one peak to find its frequency
reload(sa_2D)

print len(wavelengths)
nwavelengths = wavelengths[850:925]
nintensities = intensities[850:925]
print len(nintensities)

#Finding the maximum value of this peak and then the corresponding frequency that it is at 
peak1_freq = nwavelengths[nintensities.argmax()]
peak1_int=np.asscalar(max(nintensities))
print 'The frequency of peak 1 is', peak1_freq
print 'The amplitude of peak 1 is', peak1_int

#Adding in the plotting parameters
plt.plot(nwavelengths,nintensities[:,0])
plt.plot(peak1_freq,peak1_int,'r*')
plt.ylabel('Intensity (a.u)')
plt.xlabel('Frequency (THz)')
plt.title('Peak 1 - Spectrometer Data '+data_dir)
plt.savefig(os.path.join(data_dir, 'Peak 1 - Spectrometer Data Fit.jpg'))

In [ ]:
#Zooming in on one peak to find its frequency
reload(sa_2D)
print len(wavelengths)
n2wavelengths = wavelengths[500:575]
n2intensities = intensities[500:575]

#Finding the maximum value of this peak and then the corresponding frequency that it is at 
peak2_freq = n2wavelengths[n2intensities.argmax()]
peak2_int=np.asscalar(max(n2intensities))
print 'The frequency of peak 2 is', peak2_freq
print 'The amplitude of peak 2 is', peak2_int

#Adding in the plotting parameters
plt.plot(n2wavelengths,n2intensities[:,0])
plt.plot(peak2_freq,peak2_int,'r*')
plt.ylabel('Intensity (a.u)')
plt.xlabel('Frequency (THz)')
plt.title('Peak 2 - Spectrometer Data '+data_dir)
plt.savefig(os.path.join(data_dir, 'Peak 2 - Spectrometer Data Fit.jpg'))

# Now using the linewidth calculated and the peaks chosen for the FSR, we can find the Finesse and Cavity Length

In [ ]:
c = 3.e8
peak1_freq = 455.e12
peak2_freq = 492.e12
FSR_freq = (peak2_freq-peak1_freq) #FSR (in Hz)
dv = 1.8*1.e9 #Average of the linewidths found for the cavity (in Hz)
F= FSR_freq/dv #Finesse 
L = c/(2*FSR_freq) #Cavity Length

print L
print 'The Finesse is', round(F,2)
print 'The Cavity Length is', round(L*1.e6,2), 'um'

# text_file = open(os.path.join(data_dir,'Characterization_Results.txt'), 'w')
# text_file.write("Mirror Characterization" )
# text_file.write('\n' "Spectrometer Data: %s" % data_dir)
# text_file.write('\n' "Oscilloscope Data Average of: %s" %indir)
# text_file.write('\n' "Processed and Annealed")
# text_file.write('\n' "The Linewidth is: %s" % (dv/1.e9))
# text_file.write(" GHz")
# text_file.write('\n' "The Cavity Length is: %s" % round(L*1.e6,2))
# text_file.write(" um")
# text_file.write('\n' "The Finesse is: %s" % (round(F,2)))
# text_file.close()